In [7]:
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib oauth2client 

Requirement already up-to-date: google-api-python-client in /Users/jkanclerz/myplace/dydaktyka/hurtownie-danych/.venv/lib/python3.8/site-packages (1.12.8)
Requirement already up-to-date: google-auth-httplib2 in /Users/jkanclerz/myplace/dydaktyka/hurtownie-danych/.venv/lib/python3.8/site-packages (0.0.4)
Requirement already up-to-date: google-auth-oauthlib in /Users/jkanclerz/myplace/dydaktyka/hurtownie-danych/.venv/lib/python3.8/site-packages (0.4.2)
     |████████████████████████████████| 98 kB 1.6 MB/s eta 0:00:01
You should consider upgrading via the '/Users/jkanclerz/myplace/dydaktyka/hurtownie-danych/.venv/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


## Google analytics api

### docs
https://developers.google.com/analytics/devguides/reporting/core/v4/rest/
https://oauth2client.readthedocs.io/en/latest/source/oauth2client.service_account.html

### metrics
https://ga-dev-tools.appspot.com/dimensions-metrics-explorer/

In [148]:
key_content = '''

'''

In [149]:
from apiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

import json

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
VIEW_ID = ''


def initialize_analyticsreporting():
    credentials = ServiceAccountCredentials.from_json_keyfile_dict(json.loads(key_content), SCOPES)
    analytics = build('analyticsreporting', 'v4', credentials=credentials)

    return analytics


def get_report(analytics):
    return analytics.reports().batchGet(
      body={
        'reportRequests': [
        {
          'viewId': VIEW_ID,
          'dateRanges': [{
              'startDate': '2021-01-01',
              'endDate': '2021-01-01'
          }],
          'metrics': [{
              'expression': 'ga:sessions'
          }],
          'dimensions': [
              {
                  'name': 'ga:date'
              },
              {
                  'name': 'ga:hour'
              },
          ]
        }]
      }
  ).execute()



In [150]:
from pprint import pprint as pp

In [151]:
analytics = initialize_analyticsreporting()
response = get_report(analytics)


In [152]:
report = response.get('reports', [])[0]
rows = [row for row in report.get('data', {}).get('rows', [])]

In [153]:
visits = [("".join(row.get("dimensions", [])), row.get('metrics')[0].get('values')[0]) for row in rows]

In [154]:
visits

[('2021010102', '1'), ('2021010111', '2'), ('2021010113', '1')]

```sql
DROP TABLE IF EXISTS visits;
CREATE TABLE visits (
  id serial PRIMARY KEY,
  datetime_id INTEGER NOT NULL UNIQUE,
  visitors_count INTEGER NOT NULL,
  CONSTRAINT fk_datetime FOREIGN KEY(datetime_id) REFERENCES datetime_dimension(id)
  )
```

In [155]:
import psycopg2

In [156]:
def dest():
    connection = psycopg2.connect(user="warehouse",
                                  password="warehouse",
                                  host="127.0.0.1",
                                  port="5432",
                                  database="warehouse")
    return connection
    

In [108]:
conn = dest()
c = conn.cursor()
c.executemany('''
    Insert into visits (datetime_id, visitors_count) VALUES (%s,%s)
    ON CONFLICT 
    ON CONSTRAINT visits_datetime_id_key DO NOTHING
''', visits)

conn.commit()

## ETL

### Extract
- Visits from google for specific dates

### Transform 
??

### Load
Insert into warehouse